In [27]:
from foundations.helpers import scale_input_theory
from brian2 import *
from foundations.helpers import scale_to_freq
from foundations.make_dynamic_experiments import make_dynamic_experiments
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from models.currentmodel import Barrel_PC, Barrel_IN
from foundations.MI_calculations import analyze_exp
from foundations.helpers import make_spiketrain
%matplotlib inline

In [28]:
qon_qoff_type = 'balanced'
baseline = 0  
theta = 0     
factor_ron_roff = 2    
tau_PC = 200
ron_PC = 1./(tau_PC*(1+factor_ron_roff))
roff_PC = factor_ron_roff*ron_PC
mean_firing_rate = (0.1)/1000
duration = 100000
duration2 = 1000
sampling_rate = 5   
scales = {'CC_PC':19, 'DC_PC':30, 'CC_IN':17, 'DC_IN':6}
dt = 1/sampling_rate 
defaultclock.dt = dt*ms
on_off_ratio = 1.5
target_PC = 1.4233
scale_list = np.append([1], np.arange(2.5, 302.5, 2.5))

In [ ]:
#Getting the input, generating the same one for both pyramidal cell and interneuron
#also doing the scaling the same
[input_theory, dynamic_theory, hidden_state] = make_dynamic_experiments(qon_qoff_type, baseline, tau_PC, factor_ron_roff, mean_firing_rate, sampling_rate, duration2)
inj_input = scale_input_theory(input_theory, 'current', 0, 18, dt)

In [ ]:
#The feedforward simulation
start_scope()


PC = Barrel_PC(dt=dt)
PC2 = Barrel_PC(dt=dt)
IN = Barrel_IN(dt=dt, Input = False)

PC_neuron2 = PC2.getNeurongroup()
PC_neuron = PC.getNeurongroup()
IN_neuron = IN.getNeurongroup()

param = PC.getNamespace().update(IN.getNamespace())                                
syn_forward = Synapses(PC_neuron, IN_neuron, on_pre='''v += 50*mV''', namespace = param)
syn_forward.connect()

syn_backward = Synapses(IN_neuron, PC_neuron, delay = 0.01*ms, on_pre='''v += -50*mV''', namespace = param )
syn_backward.connect()

PC_M2 = StateMonitor(PC_neuron2, 'v', record=True)                                
PC_M = StateMonitor(PC_neuron, 'v', record=True)
IN_M = StateMonitor(IN_neuron, 'v', record=True)

PC_S2 = SpikeMonitor(PC_neuron2)                                
PC_S = SpikeMonitor(PC_neuron)
IN_S = SpikeMonitor(IN_neuron)

                                 
run(duration2*ms)

In [ ]:
figure(figsize=(10, 10))
subplot(221)
plot(PC_M2.t/ms, PC_M2.v[0], label='pre_PC')
legend()

subplot(222)
plot(IN_M.t/ms, IN_M.v[0], label="pre_IN")
legend()


subplot(223)
plot(PC_M.t/ms, PC_M.v[0], label='post_PC')
legend()

subplot(224)
plot(PC_M.t/ms, PC_M.v[0], label="post_PC")
legend()
xlabel('Time (ms)');

In [ ]:
figure(figsize=(10, 4))
subplot(311)
plot(PC_M2.t/ms, PC_M2.v[0], label='pre_PC')
legend()
subplot(312)
plot(IN_M.t/ms, IN_M.v[0], label="pre_IN")
legend()
subplot(313)
plot(PC_M.t/ms, PC_M.v[0], label="post_PC")
legend()

## Estimating parameters

In [ ]:
#The feedforward simulation
start_scope()

PC = Barrel_PC(dt=dt)
PC2 = Barrel_PC(dt=dt)
IN = Barrel_IN(dt=dt)

PC_neuron2 = PC2.getNeurongroup()
PC_neuron = PC.getNeurongroup()
IN_neuron = IN.getNeurongroup()

param = PC.getNamespace().update(IN.getNamespace())                                
syn = Synapses(PC_neuron, IN_neuron, model = 'w : 1', on_pre='''v += w*mV''', namespace = param)
syn.connect()
syn.w = 10

syn_backward = Synapses(IN_neuron, PC_neuron, model = 'w:1', on_pre='''v += w*mV''', namespace = param )
syn_backward.connect()

PC_M2 = StateMonitor(PC_neuron2, 'v', record=True)                                
PC_M = StateMonitor(PC_neuron, 'v', record=True)
IN_M = StateMonitor(IN_neuron, 'v', record=True)

PC_S2 = SpikeMonitor(PC_neuron2)                                
PC_S = SpikeMonitor(PC_neuron)
IN_S = SpikeMonitor(IN_neuron)

df = pd.DataFrame()
#setting the dataframe
weigths = [-0.5,-10,-50,-200]

store()
for weigth in weigths: 
    restore()
    
    syn_backward.w = [weigth]
    run(duration*ms)
    
    IN_spike  = make_spiketrain(IN_S,duration,dt)
    PC_spike  = make_spiketrain(PC_S,duration,dt)
    PC_spike2 = make_spiketrain(PC_S2, duration, dt)
    
    result_PC  = analyze_exp('PC_post',ron_PC, roff_PC, hidden_state, input_theory, dt, 0, PC_spike)
    result_IN  = analyze_exp('IN'     , ron_PC, roff_PC, hidden_state, input_theory, dt, 0, IN_spike)
    result_PC2 = analyze_exp('PC_pre' , ron_PC, roff_PC, hidden_state, input_theory, dt, 0, PC_spike2)
    
    frames = [df,result_PC, result_IN, result_PC2]
    df = pd.concat(frames)
    

In [ ]:
df